# Item Based Collaborative Filtering

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/BERT_modeling/labeled_dataset.csv')

In [ ]:
df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,rank,company,name1,name2,category,volume,...,score,real_date,buy,real_review,split_sentences,sentence,words,normalized,polarity,sentiment
0,0,0,0,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",상한 겁니까,상한 걸 ㅂ니까,상한 겁니까,-6.0,negative
1,1,1,1,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",구매확정합니다,구매 확정 하 ㅂ니다,구매확정합니다,0.0,positive
2,2,2,2,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",올바른 상품이 왔더니 이건 도대체 다시는 여기 거 안 먹습니다,올바르 ㄴ 상품 이 오 았 더니 이 것 은 도대체 다시 는 여기 그 어 안 먹 습니다,올바른 상품이 왔더니 이건 도대체 다시는 여기 거 안 먹습니다,4.5,positive
3,3,3,3,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",올바른 상태,올바르 ㄴ 상태,올바른 상태,3.0,positive
4,4,4,4,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",저번에 시킨 박스는 제품 상하 뒤집지 말아달란 스티커와 올바른 박스,저번 에 시키 ㄴ 박스 는 제품 상하 뒤집 지 말 아 달 란 스티커 와 올바르 ㄴ 박스,저번에 시킨 박스는 제품 상하 뒤집지 말아달란 스티커와 올바른 박스,3.0,positive


### 1. 유저 아이디 생성

In [ ]:
# Generate random user IDs
user_ids = random.choices(range(1, 10001), k=len(df))

# Add user_id column to the dataframe
df['user_id'] = user_ids
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,rank,company,name1,name2,category,volume,...,real_date,buy,real_review,split_sentences,sentence,words,normalized,polarity,sentiment,user_id
0,0,0,0,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,...,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",상한 겁니까,상한 걸 ㅂ니까,상한 겁니까,-6.0,negative,637
1,1,1,1,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,...,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",구매확정합니다,구매 확정 하 ㅂ니다,구매확정합니다,0.0,positive,8584
2,2,2,2,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,...,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",올바른 상품이 왔더니 이건 도대체 다시는 여기 거 안 먹습니다,올바르 ㄴ 상품 이 오 았 더니 이 것 은 도대체 다시 는 여기 그 어 안 먹 습니다,올바른 상품이 왔더니 이건 도대체 다시는 여기 거 안 먹습니다,4.5,positive,1977
3,3,3,3,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,...,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",올바른 상태,올바르 ㄴ 상태,올바른 상태,3.0,positive,9609
4,4,4,4,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,...,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",저번에 시킨 박스는 제품 상하 뒤집지 말아달란 스티커와 올바른 박스,저번 에 시키 ㄴ 박스 는 제품 상하 뒤집 지 말 아 달 란 스티커 와 올바르 ㄴ 박스,저번에 시킨 박스는 제품 상하 뒤집지 말아달란 스티커와 올바른 박스,3.0,positive,6079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305255,305255,305308,305634,147662,121,주로,골목막걸리 프리미엄 12,골목막걸리 프리미엄 12,막걸리/탁주,350,...,22.08.21.,우리술한잔,맛있네요 택배 포장 걱정했는데\n너무 만족으럽게 포장해 주셔서 잘 받았습니다.\n다...,"['맛있네요', '택배 포장 걱정했는데\n너무 만족으럽게 포장해 주셔서 잘 받았습니...",또르르,또르르,또르르,1.5,positive,9210
305256,305256,305309,305635,147663,121,주로,골목막걸리 프리미엄 12,골목막걸리 프리미엄 12,막걸리/탁주,350,...,22.06.07.,술마켓,꼼콤하게 잘 왔구요 걸쭉하니 만납니다.,"['꼼콤하게 잘 왔구요', '걸쭉하니 만납니다.']",꼼콤하게 잘 왔고요,꼬 ㅁ 콤 하 게 잘 오 았 고요,꼼콤하게 잘 왔고요,3.0,positive,5382
305257,305257,305310,305636,147663,121,주로,골목막걸리 프리미엄 12,골목막걸리 프리미엄 12,막걸리/탁주,350,...,22.06.07.,술마켓,꼼콤하게 잘 왔구요 걸쭉하니 만납니다.,"['꼼콤하게 잘 왔구요', '걸쭉하니 만납니다.']",걸쭉하니 만납니다,걸쭉 하 니 만나 ㅂ니다,걸쭉하니 만납니다,0.0,positive,3988
305258,305258,305311,305637,147664,121,주로,골목막걸리 프리미엄 12,골목막걸리 프리미엄 12,막걸리/탁주,350,...,22.07.06.,술마켓,따봉 맛있어요 얼음 넣으면 신의한수,"['따봉 맛있어요', '얼음 넣으면 신의한수']",따봉 맛있어요,따 아 봉 맛있 어요,따봉 맛있어요,6.0,positive,8585


### 2. product-user 매트릭스 생성

In [ ]:
df['category'].unique()

array(['막걸리/탁주', '리큐르', '과실주/와인', '약주/청주', '소주', '증류주'], dtype=object)

In [ ]:
cols = ['user_id', 'category', 'name2', 'score', 'polarity', 'sentiment']
df = df[cols]
df.head()

,user_id,category,name2,score,polarity,sentiment
0,637,막걸리/탁주,느린마을 막걸리 6도,1,-6.0,negative
1,8584,막걸리/탁주,느린마을 막걸리 6도,1,0.0,positive
2,1977,막걸리/탁주,느린마을 막걸리 6도,1,4.5,positive
3,9609,막걸리/탁주,느린마을 막걸리 6도,1,3.0,positive
4,6079,막걸리/탁주,느린마을 막걸리 6도,1,3.0,positive


In [ ]:
df1 = df[df['category'] == '막걸리/탁주']
df2 = df[df['category'] == '리큐르']
df3 = df[df['category'] == '과실주/와인']
df4 = df[df['category'] == '약주/청주']
df5 = df[df['category'] == '소주']
df6 = df[df['category'] == '증류주']

print(len(df1['name2'].unique()))
print(len(df2['name2'].unique()))
print(len(df3['name2'].unique()))
print(len(df4['name2'].unique()))
print(len(df5['name2'].unique()))
print(len(df6['name2'].unique()))

38
11
21
16
20
11


### 3. 아이템 기반 협업 필터링
* 코사인 유사도 사용하여 제품 간 유사도 측정
* 추천 서비스를 위해, 각 제품별로 유사도 상위 top 5 제품 매칭

In [ ]:
def get_top_similar_names(similarity_matrix, product_name_list, n_similar=5):
    # Get the top similar names for each name2
    similar_names_dict = {}
    for idx, name in enumerate(product_name_list):
        sim_values = similarity_matrix[idx]
        similar_indices = np.argsort(sim_values)[::-1][1:n_similar+1]
        similar_names = product_name_list[similar_indices]
        similar_names_dict[name] = similar_names

    # Return the dictionary of similar names
    return similar_names_dict

def get_similar_names_for_dataframe(df):
    average_polarity = df.groupby(['user_id', 'name2'])['polarity'].mean().reset_index()

    # Create a new data frame with the average polarity values
    new_df = pd.DataFrame({'user_id': average_polarity['user_id'], 'name2': average_polarity['name2'], 'avg_polarity': average_polarity['polarity']})

    product_usr_pivot = new_df.pivot_table('avg_polarity', index='user_id', columns='name2')
    product_usr_pivot.fillna(0, inplace=True)

    # Item-based collaborative filtering using cosine similarity
    similarity_matrix = cosine_similarity(product_usr_pivot.T, product_usr_pivot.T)

    # Calculate the top similar names for each name2
    similar_names_dict = get_top_similar_names(similarity_matrix, product_usr_pivot.columns, n_similar=5)

    # Convert the similarity dictionary to a DataFrame
    similar_names_df = pd.DataFrame.from_dict(similar_names_dict, orient='index', columns=[f"SimilarName{i}" for i in range(1, 6)])

    # Reset the index to create a 'name2' column
    similar_names_df.reset_index(inplace=True)
    similar_names_df.rename(columns={'index': 'name2'}, inplace=True)

    return similar_names_df

In [ ]:
similar_names_df1 = get_similar_names_for_dataframe(df1)
similar_names_df2 = get_similar_names_for_dataframe(df2)
similar_names_df3 = get_similar_names_for_dataframe(df3)
similar_names_df4 = get_similar_names_for_dataframe(df4)
similar_names_df5 = get_similar_names_for_dataframe(df5)
similar_names_df6 = get_similar_names_for_dataframe(df6)

print("DF1:")
print(similar_names_df1)

print("DF2:")
print(similar_names_df2)

print("DF3:")
print(similar_names_df3)

print("DF4:")
print(similar_names_df4)

print("DF5:")
print(similar_names_df5)

print("DF6:")
print(similar_names_df6)

DF1:
                 name2        SimilarName1        SimilarName2  \
0       고흥 풍양 유자막걸리 6도         이화백주 순탁주 6도   복순도가 빨간쌀 막걸리 6.5도   
1        골목막걸리 프리미엄 12         이화백주 순탁주 6도         나루 생 막걸리 6도   
2           공주알밤왕밤주 6도      복순도가 손막걸리 6.5도         이화백주 순탁주 6도   
3           금정산성막걸리 8도      복순도가 손막걸리 6.5도         나루 생 막걸리 6도   
4          나루 생 막걸리 6도         이화백주 순탁주 6도      복순도가 손막걸리 6.5도   
5          느린마을 막걸리 6도      복순도가 손막걸리 6.5도         이화백주 순탁주 6도   
6   담은 생막걸리 WHITE 6.5도         이화백주 순탁주 6도      복순도가 손막걸리 6.5도   
7         대대포(Blue) 6도         이화백주 순탁주 6도      복순도가 손막걸리 6.5도   
8              메로니아 4%      복순도가 손막걸리 6.5도          호랑이생막걸리 6도   
9       배혜정도가 우곡생주 10도         이화백주 순탁주 6도      복순도가 손막걸리 6.5도   
10     백련 생막걸리 Snow 6도         해창12도찹쌀생막걸리         이화백주 순탁주 6도   
11             별산 6.5도           표문 막걸리 6도         이화백주 순탁주 6도   
12   복순도가 빨간쌀 막걸리 6.5도         이화백주 순탁주 6도      복순도가 손막걸리 6.5도   
13      복순도가 손막걸리 6.5도         이화백주 순탁주 6도           표문 막걸리 6도   
14   

In [ ]:
# Combine the dataframes into one
similar_names_combined = pd.concat([similar_names_df1, similar_names_df2, similar_names_df3, similar_names_df4, similar_names_df5, similar_names_df6], ignore_index=True)
similar_names_combined

,name2,SimilarName1,SimilarName2,SimilarName3,SimilarName4,SimilarName5
0,고흥 풍양 유자막걸리 6도,이화백주 순탁주 6도,복순도가 빨간쌀 막걸리 6.5도,호랑이생막걸리 6도,복순도가 손막걸리 6.5도,표문 막걸리 6도
1,골목막걸리 프리미엄 12,이화백주 순탁주 6도,나루 생 막걸리 6도,담은 생막걸리 WHITE 6.5도,해창9도찹쌀생막걸리,해창12도찹쌀생막걸리
2,공주알밤왕밤주 6도,복순도가 손막걸리 6.5도,이화백주 순탁주 6도,금정산성막걸리 8도,담은 생막걸리 WHITE 6.5도,호랑이생막걸리 6도
3,금정산성막걸리 8도,복순도가 손막걸리 6.5도,나루 생 막걸리 6도,이화백주 순탁주 6도,해창12도찹쌀생막걸리,표문 막걸리 6도
4,나루 생 막걸리 6도,이화백주 순탁주 6도,복순도가 손막걸리 6.5도,붉은원숭이 10.8도,표문 막걸리 6도,선호 생막걸리 6도
...,...,...,...,...,...,...
112,신례명주 50도,미상25,금설 35도,참조은안동쏘주16.9%,담금나라 30도,제주 동백 16도
113,제주 동백 16도,금설 35도,참조은안동쏘주16.9%,미상25,토끼소주 선비 진 48도,감홍로 40도
114,참조은안동쏘주16.9%,금설 35도,미상25,담금나라 30도,감홍로 40도,추사40
115,추사40,금설 35도,참조은안동쏘주16.9%,미상25,담금나라 30도,감홍로 40도


In [ ]:
similar_names_combined.to_csv('/content/drive/MyDrive/BERT_modeling/item_based_recommended_dataset.csv')